In [ ]:
import time
from tqdm import tqdm

import cv2
import numpy as np

from ultralytics import YOLO

import supervision as sv

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
model = YOLO('yolov8x.pt')

In [ ]:
video = 'videos/traffic.mp4'

In [ ]:
video_info = sv.VideoInfo.from_video_path(video)
video_info.resolution_wh

In [ ]:
generator = sv.get_video_frames_generator(video_info)
iterator = iter(generator)
frame = next(iterator)
frame.shape

In [ ]:
plt.imshow(frame[:,:,::-1])
plt.show()

In [ ]:
polygons = [
    np.array([[600, 680], [927, 680], [851, 950], [42, 950]]),
    np.array([[987, 680], [1350, 680], [1893, 950], [1015, 950]])
]

zones = [sv.PolygonZone(polygon=polygon, frame_resolution_wh=video_info.resolution_wh) for polygon in polygons]

In [ ]:
colors = sv.ColorPalette.default()

In [ ]:
zone_annotators = [
    sv.PolygonZoneAnnotator(zone=zone, color=colors.by_idx(index), thickness=6, text_thickness=12, text_scale=4)
    for index, zone in enumerate(zones)
]

In [ ]:
results = model(frame, imgsz=1280, verbose=False, show=False, device='cuda:0')[0]
detections = sv.Detections.from_yolov8(results)


In [ ]:
for zone, zone_annotator, box_annotator in zip(zones, zone_annotators, box_annotators):
    
    mask = zone.trigger(detections=detections)
    detections_filtered = detections[mask]
    frame = box_annotator.annotate(scene=frame, detections=detections_filtered, skip_label=True)
    frame = zone_annotator.annotate(scene=frame)

plt.imshow(frame[:,:,::-1])
plt.show()

In [ ]:
plt.imshow(zone.mask)
plt.show()
zone.current_count

In [ ]:
def process_frame(frame: np.ndarray, i) -> np.ndarray:
    results = model(frame, imgsz=1280, verbose=False, show=False, device='cuda:0')[0]
    detections = sv.Detections.from_yolov8(results)
    for zone, zone_annotator, box_annotator in zip(zones, zone_annotators, box_annotators):
        mask = zone.trigger(detections=detections)
        detections_filtered = detections[mask]
        frame = box_annotator.annotate(scene=frame, detections=detections_filtered, skip_label=True)
        frame = zone_annotator.annotate(scene=frame)
    pbar.update(1)

    return frame


In [ ]:
filehead = video.split('/')[-1]
OUT_PATH = "out-" + filehead

In [ ]:
with tqdm(total=video_info.total_frames-1) as pbar:
    sv.process_video(source_path=video, target_path=OUT_PATH, callback=process_frame)
